In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup

# Find Performance Data

1. Batting: `https://stats.espncricinfo.com/ci/engine/player/<:playerID>.html?class=2;template=results;type=batting;view=innings`
2. Bowling: `https://stats.espncricinfo.com/ci/engine/player/<:playerID>.html?class=2;template=results;type=bowling;view=innings`
3. AllRound: `https://stats.espncricinfo.com/ci/engine/player/<:playerID>.html?class=2;template=results;type=allround;view=innings`

In [2]:
player_data = pd.read_csv('player_{team_name_id_role}.csv')
player_data

team_name           player_name  player_id          player_role
0    Afghanistan  Hashmatullah Shahidi     440970     Top order Batter
1    Afghanistan    Rahmanullah Gurbaz     974087  Wicketkeeper Batter
2    Afghanistan        Ibrahim Zadran     921509       Opening Batter
3    Afghanistan          Reyaz Hassan    1061090               Batter
4    Afghanistan   Rahmat Shah Zurmati     533956           Allrounder
..           ...                   ...        ...                  ...
145    Sri Lanka      Dunith Wellalage    1152427               Bowler
146    Sri Lanka         Kasun Rajitha     499594               Bowler
147    Sri Lanka   Matheesha Pathirana    1194795               Bowler
148    Sri Lanka         Lahiru Kumara     784375               Bowler
149    Sri Lanka    Dilshan Madushanka     793007               Bowler

[150 rows x 4 columns]

## Bowling Data Extraction

In [7]:
player_id = "26421"

def fetch_performance_data(player_id):
    url = f"https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class=2;template=results;type=bowling;view=innings"


    try:
        fetched_data = get(url)

        soup = BeautifulSoup(fetched_data.text, 'html.parser')

        # find all caption tags where text is "Innings by innings list"
        captions = soup.find_all('caption')
        table = None
        for caption in captions:
            if caption.text == "Innings by innings list":
                table = caption.find_parent('table')

        # Select all rows with class data1
        rows = table.find_all('tr', class_='data1')
        raw_data = []

        for row in rows:
            cols = row.find_all('td')
            row_data = [player_id]
            for col in cols:
                # If col has a links, get the text of the last link
                if col.find('a'):
                    row_data.append(col.find_all('a')[-1].text)
                else:
                    if col.text != '':
                        row_data.append(col.text)

            raw_data.append(row_data)
    except:
        print(f"Skipping player {player_id}")
        return [None for i in range(12)]

    return raw_data

# raw_data = fetch_performance_data(player_id)
# test_df = pd.DataFrame(raw_data, columns=['player_id', 'overs', 'maidens', 'runs', 'wickets', 'economy', 'position', 'innings', 'opposition', 'ground', 'start_date', 'odi_number'])


116

In [8]:
# for player_id in player_data['player_id']: find_performance_data(player_id) and merge with df

performane_raw_data = []
for player_id in player_data['player_id']:
    try:
        performane_raw_data.extend(fetch_performance_data(player_id))
    except:
        print(f"Skipping player {player_id}")

len(performane_raw_data)


9331

In [17]:
column_names = ['player_id', 'overs', 'maidens', 'runs', 'wickets', 'economy', 'position', 'innings', 'opposition', 'ground', 'start_date', 'odi_number']

bowling_df = pd.DataFrame(performane_raw_data, columns=column_names)
bowling_df

player_id overs maidens runs wickets economy position innings  \
0        440970   DNB       -    -       -       -        -       1   
1        440970   DNB       -    -       -       -        -       1   
2        440970   DNB       -    -       -       -        -       1   
3        440970   2.0       0   17       0    8.50        6       2   
4        440970   DNB       -    -       -       -        -       2   
...         ...   ...     ...  ...     ...     ...      ...     ...   
9326     793007  10.0       0   86       2    8.60        2       1   
9327     793007   9.2       0   60       2    6.42        2       2   
9328     793007   9.0       2   38       3    4.22        2       2   
9329     793007   9.4       1   49       4    5.06        1       1   
9330     793007   5.0       0   37       0    7.40        1       1   

        opposition        ground   start_date  odi_number  
0            Kenya       Sharjah   2 Oct 2013  ODI # 3417  
1            Kenya       Sharjah   4 Oct 2013  ODI # 3418  
2        Hong Kong  Kuala Lumpur   1 May 2014  ODI # 3487  
3           U.A.E.  Kuala Lumpur   2 May 2014  ODI # 3488  
4         Zimbabwe      Bulawayo  18 Jul 2014  ODI # 3503  
...            ...           ...          ...         ...  
9326  South Africa         Delhi   7 Oct 2023  ODI # 4661  
9327      Pakistan     Hyderabad  10 Oct 2023  ODI # 4665  
9328     Australia       Lucknow  16 Oct 2023  ODI # 4671  
9329   Netherlands       Lucknow  21 Oct 2023  ODI # 4676  
9330       England     Bengaluru  26 Oct 2023  ODI # 4682  

[9331 rows x 12 columns]

In [18]:
bowling_df.to_csv('bowling_data.csv', index=False)

## Batting Data Extraction

In [28]:
# column_names = ["player_id", "Runs",	"Mins",	"BF", "4s", "6s", "SR", "Pos", "Dismissal", "Inns","Opposition", "Ground", "Start", "odi_number"]

# player_id = "253802"

def fetch_batting_data(player_id):
    url = f"https://stats.espncricinfo.com/ci/engine/player/{player_id}.html?class=2;template=results;type=batting;view=innings"

    try:
        fetched_data = get(url)

        soup = BeautifulSoup(fetched_data.text, 'html.parser')

        # find all caption tags where text is "Innings by innings list"
        captions = soup.find_all('caption')
        table = None
        for caption in captions:
            if caption.text == "Innings by innings list":
                table = caption.find_parent('table')

        # Select all rows with class data1
        rows = table.find_all('tr', class_='data1')
        raw_data = []

        for row in rows:
            cols = row.find_all('td')
            row_data = [player_id]
            for col in cols:
                # If col has a links, get the text of the last link
                if col.find('a'):
                    row_data.append(col.find_all('a')[-1].text)
                else:
                    if col.text != '':
                        row_data.append(col.text)

            raw_data.append(row_data)
    except Exception as e:
        print(f"Skipping player {player_id}")

    return raw_data

# raw_data = fetch_batting_data(player_id)
# len(raw_data)
# df = pd.DataFrame(raw_data, columns=column_names)
# df

player_id  Runs Mins   BF 4s 6s      SR Pos Dismissal Inns   Opposition  \
0      253802    12   33   22  1  0   54.54   2       lbw    1    Sri Lanka   
1      253802    37   82   67  6  0   55.22   2    caught    2    Sri Lanka   
2      253802    25   40   38  4  0   65.78   1   run out    1    Sri Lanka   
3      253802    54   87   66  7  0   81.81   1    bowled    1    Sri Lanka   
4      253802    31   45   46  3  1   67.39   1       lbw    2    Sri Lanka   
..        ...   ...  ...  ... .. ..     ...  ..       ...  ...          ...   
281    253802    85  171  116  6  0   73.27   3    caught    2    Australia   
282    253802   55*   64   56  6  0   98.21   3   not out    2  Afghanistan   
283    253802    16   31   18  3  0   88.88   3    caught    2     Pakistan   
284    253802  103*  113   97  6  4  106.18   3   not out    2   Bangladesh   
285    253802    95  154  104  8  2   91.34   3    caught    2  New Zealand   

            Ground        Start  odi_number  
0         Dambulla  18 Aug 2008  ODI # 2742  
1         Dambulla  20 Aug 2008  ODI # 2745  
2    Colombo (RPS)  24 Aug 2008  ODI # 2750  
3    Colombo (RPS)  27 Aug 2008  ODI # 2755  
4    Colombo (RPS)  29 Aug 2008  ODI # 2756  
..             ...          ...         ...  
281        Chennai   8 Oct 2023  ODI # 4662  
282          Delhi  11 Oct 2023  ODI # 4666  
283      Ahmedabad  14 Oct 2023  ODI # 4669  
284           Pune  19 Oct 2023  ODI # 4674  
285     Dharamsala  22 Oct 2023  ODI # 4678  

[286 rows x 14 columns]

In [29]:
# for player_id in player_data['player_id']: find_performance_data(player_id) and merge with df

batting_raw_data = []
for player_id in player_data['player_id']:
    try:
        batting_raw_data.extend(fetch_batting_data(player_id))
    except:
        print(f"Skipping player {player_id}")

len(batting_raw_data)


9331

In [30]:
batting_df = pd.DataFrame(batting_raw_data, columns=column_names)
batting_df

player_id  Runs Mins  BF 4s 6s      SR Pos Dismissal Inns    Opposition  \
0        440970     0    -   3  0  0    0.00   3    caught    2         Kenya   
1        440970   13*    -  19  0  0   68.42   5   not out    2         Kenya   
2        440970   38*    -  67  4  0   56.71   4   not out    2     Hong Kong   
3        440970     3    -   7  0  0   42.85   4       lbw    1        U.A.E.   
4        440970    15   21  22  2  0   68.18   4    caught    1      Zimbabwe   
...         ...   ...  ...  .. .. ..     ...  ..       ...  ...           ...   
9326     793007    4*    4   2  1  0  200.00  11   not out    2  South Africa   
9327     793007   DNB    -   -  -  -       -   -         -    1      Pakistan   
9328     793007    0*   12   6  0  0    0.00  11   not out    1     Australia   
9329     793007   DNB    -   -  -  -       -   -         -    2   Netherlands   
9330     793007  TDNB    -   -  -  -       -   -         -    -       England   

            Ground        Start  odi_number  
0          Sharjah   2 Oct 2013  ODI # 3417  
1          Sharjah   4 Oct 2013  ODI # 3418  
2     Kuala Lumpur   1 May 2014  ODI # 3487  
3     Kuala Lumpur   2 May 2014  ODI # 3488  
4         Bulawayo  18 Jul 2014  ODI # 3503  
...            ...          ...         ...  
9326         Delhi   7 Oct 2023  ODI # 4661  
9327     Hyderabad  10 Oct 2023  ODI # 4665  
9328       Lucknow  16 Oct 2023  ODI # 4671  
9329       Lucknow  21 Oct 2023  ODI # 4676  
9330     Bengaluru  26 Oct 2023  ODI # 4682  

[9331 rows x 14 columns]

In [31]:
batting_df.to_csv('batting_data.csv', index=False)

In [32]:
batting_df['player_id'].value_counts()

253802     286
56029      261
34102      256
56143      244
56025      225
          ... 
721041       4
819507       3
1139520      2
35281        2
379887       2
Name: player_id, Length: 150, dtype: int64